In [ ]:
# Standard installs
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft bitsandbytes
!pip install datasets # If loading from Hugging Face Hub
!pip install --upgrade bitsandbytes
import torch
from unsloth import FastLanguageModel
from datasets import load_dataset
# Import necessary classes from transformers and peft
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, Trainer
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-4ekp8ixc/unsloth_b5c8a0a5024f42099c90a0f902888619
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-4ekp8ixc/unsloth_b5c8a0a5024f42099c90a0f902888619
  Resolved https://github.com/unslothai/unsloth.git to commit 7a8f99e1890213cdd01a3ab6c3e13174a96e8220
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
# Load model: TinyLlama
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# Prepare model for LoRA
model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)

# Load dataset
dataset = load_dataset("mlabonne/orpo-dpo-mix-40k-flat", split="train[:500]")

# Correct tokenization
def tokenize_function(example):
    model_inputs = tokenizer(
        example["prompt"],
        truncation=True,
        padding="max_length",
        max_length=512,
    )
    model_inputs["labels"] = model_inputs["input_ids"].copy()
    return model_inputs

dataset = dataset.map(tokenize_function)


# Initial fine-tuning (first phase)
training_args_first = TrainingArguments(
    output_dir="./tinyllama_checkpoint_1",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    num_train_epochs=1,
    save_steps=10,
    logging_steps=5,
    fp16=True,
    bf16=False,
    save_total_limit=2,
)

trainer_first = Trainer(
    model=model,
    args=training_args_first,
    train_dataset=dataset,
    tokenizer=tokenizer,
)

trainer_first.train()

# Save after initial phase
model.save_pretrained("./tinyllama_checkpoint_1/final_model")
tokenizer.save_pretrained("./tinyllama_checkpoint_1/final_model")

# Load the previously saved checkpoint
model = AutoModelForCausalLM.from_pretrained(
    "./tinyllama_checkpoint_1/final_model",
    device_map="auto",
    load_in_4bit=True,
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained(
    "./tinyllama_checkpoint_1/final_model",
    trust_remote_code=True,
)

# Prepare again for LoRA
model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)

# Continued training (second phase)
training_args_second = TrainingArguments(
    output_dir="./tinyllama_checkpoint_2",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    num_train_epochs=1,
    save_steps=10,
    logging_steps=5,
    fp16=True,
    bf16=False,
    save_total_limit=2,
)

trainer_second = Trainer(
    model=model,
    args=training_args_second,
    train_dataset=dataset,
    tokenizer=tokenizer,
)

trainer_second.train()

# Save final continued model
model.save_pretrained("./tinyllama_checkpoint_2/final_model")
tokenizer.save_pretrained("./tinyllama_checkpoint_2/final_model")

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

<ipython-input-6-762e2013a4e6>:60: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_first = Trainer(


Step,Training Loss
5,9.658600
10,9.757300
15,8.673600
20,7.873800
25,6.876900
30,4.992700
35,5.312700
40,3.994800
45,2.848800
50,2.324700


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
<ipython-input-6-762e2013a4e6>:112: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_second = Trainer(
/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Line

Step,Training Loss
5,9.658600
10,9.755700
15,8.651300
20,7.837600
25,6.795600
30,4.962200
35,5.241200
40,3.959600
45,2.725100
50,2.326100


('./tinyllama_checkpoint_2/final_model/tokenizer_config.json',
 './tinyllama_checkpoint_2/final_model/special_tokens_map.json',
 './tinyllama_checkpoint_2/final_model/tokenizer.model',
 './tinyllama_checkpoint_2/final_model/added_tokens.json',
 './tinyllama_checkpoint_2/final_model/tokenizer.json')